In [1]:
import random

def create_non_minimal_diverse_automata(num_automata):
    automata_database = []
    
    for i in range(1, num_automata + 1):
        alphabet = ['a', 'b', 'c']
        
        num_states = 4 + i % 3
        states = [f's{j}' for j in range(num_states)]
        # Introduce at least one redundant state explicitly
        redundant_state = f's{num_states}'
        states.append(redundant_state)
        
        transitions = {}
        for state in states:
            transitions[state] = {}
            if i % 2 == 0:
                transitions[state]['a'] = states[(states.index(state) + 1) % len(states)]
                transitions[state]['b'] = states[(states.index(state) - 1) % len(states)]
                transitions[state]['c'] = state  # Adding self-loop on 'c' (could be redundant)
            else:
                transitions[state]['a'] = state
                transitions[state]['b'] = states[(states.index(state) + 2) % len(states)]
                transitions[state]['c'] = states[(states.index(state) - 1) % len(states)]
        # Ensuring redundancy in transitions
        transitions[redundant_state]['a'] = redundant_state  # Self-loop
        transitions[redundant_state]['b'] = redundant_state  # Self-loop
        transitions[redundant_state]['c'] = redundant_state  # Self-loop

        start_state = 's0'
        # Adding redundancy in accept states
        accept_states = [states[-2], redundant_state]
        
        automaton = {
            'Name': f'Automaton_{i}',
            'States': states,
            'Alphabet': alphabet,
            'Transitions': transitions,
            'Start State': start_state,
            'Accept States': accept_states
        }
        
        automata_database.append(automaton)
    
    return automata_database

# Generate 10 non-minimal diverse automata
automata_list = create_non_minimal_diverse_automata(10)

# Display the structure of the first automaton for verification
print(automata_list[0])


{'Name': 'Automaton_1', 'States': ['s0', 's1', 's2', 's3', 's4', 's5'], 'Alphabet': ['a', 'b', 'c'], 'Transitions': {'s0': {'a': 's0', 'b': 's2', 'c': 's5'}, 's1': {'a': 's1', 'b': 's3', 'c': 's0'}, 's2': {'a': 's2', 'b': 's4', 'c': 's1'}, 's3': {'a': 's3', 'b': 's5', 'c': 's2'}, 's4': {'a': 's4', 'b': 's0', 'c': 's3'}, 's5': {'a': 's5', 'b': 's5', 'c': 's5'}}, 'Start State': 's0', 'Accept States': ['s4', 's5']}


In [26]:
automata_list[0]

{'Name': 'Automaton_1',
 'States': ['s0', 's1', 's2', 's3', 's4', 's5'],
 'Alphabet': ['a', 'b', 'c'],
 'Transitions': {'s0': {'a': 's0', 'b': 's2', 'c': 's5'},
  's1': {'a': 's1', 'b': 's3', 'c': 's0'},
  's2': {'a': 's2', 'b': 's4', 'c': 's1'},
  's3': {'a': 's3', 'b': 's5', 'c': 's2'},
  's4': {'a': 's4', 'b': 's0', 'c': 's3'},
  's5': {'a': 's5', 'b': 's5', 'c': 's5'}},
 'Start State': 's0',
 'Accept States': ['s4', 's5']}

In [4]:
import xml.etree.ElementTree as ET
import random

def create_xml_string(automata_list):
    root = ET.Element("structure")
    root.set("type", "fa")
    root.set("id", "1")

    for automaton in automata_list:
        automaton_tag = ET.SubElement(root, "automaton")
        automaton_tag.set("name", automaton['Name'])
        
        # Adding states
        for i, state in enumerate(automaton['States']):
            state_tag = ET.SubElement(automaton_tag, "state")
            state_tag.set("id", str(i))
            state_tag.set("name", state)
            x, y = random.randint(100, 400), random.randint(100, 400)  # Random coordinates
            ET.SubElement(state_tag, "x").text = str(x)
            ET.SubElement(state_tag, "y").text = str(y)
            if state in automaton['Accept States']:
                ET.SubElement(state_tag, "final")
            if state == automaton['Start State']:
                ET.SubElement(state_tag, "initial")
        
        # Adding transitions
        state_id_map = {state: i for i, state in enumerate(automaton['States'])}
        for state, transitions in automaton['Transitions'].items():
            for input_symbol, target_state in transitions.items():
                transition_tag = ET.SubElement(automaton_tag, "transition")
                ET.SubElement(transition_tag, "from").text = str(state_id_map[state])
                ET.SubElement(transition_tag, "to").text = str(state_id_map[target_state])
                ET.SubElement(transition_tag, "read").text = input_symbol

    # Convert to ElementTree
    tree = ET.ElementTree(root)
    # Generate the XML string
    import io
    xml_string_io = io.StringIO()
    tree.write(xml_string_io, encoding='unicode')
    return xml_string_io.getvalue()

# Generate XML string for the automata
xml_content = create_xml_string(automata_list[:1])
xml_content

'<structure type="fa" id="1"><automaton name="Automaton_1"><state id="0" name="s0"><x>363</x><y>398</y><initial /></state><state id="1" name="s1"><x>304</x><y>116</y></state><state id="2" name="s2"><x>261</x><y>238</y></state><state id="3" name="s3"><x>397</x><y>157</y></state><state id="4" name="s4"><x>326</x><y>170</y><final /></state><state id="5" name="s5"><x>395</x><y>394</y><final /></state><transition><from>0</from><to>0</to><read>a</read></transition><transition><from>0</from><to>2</to><read>b</read></transition><transition><from>0</from><to>5</to><read>c</read></transition><transition><from>1</from><to>1</to><read>a</read></transition><transition><from>1</from><to>3</to><read>b</read></transition><transition><from>1</from><to>0</to><read>c</read></transition><transition><from>2</from><to>2</to><read>a</read></transition><transition><from>2</from><to>4</to><read>b</read></transition><transition><from>2</from><to>1</to><read>c</read></transition><transition><from>3</from><to>3</

In [5]:
def save_xml_to_file(xml_content, file_name):
    with open(file_name, 'w', encoding='utf-8') as file:
        file.write(xml_content)

# Assuming 'xml_content' is the XML string you want to save
file_name = "automaton.jff"
save_xml_to_file(xml_content, file_name)

print(f"XML content has been saved to {file_name}")

XML content has been saved to automaton.jff


In [11]:
import xml.etree.ElementTree as ET
import random

def prettify(element, indent='  '):
    queue = [(0, element)]  # (level, element)
    while queue:
        level, element = queue.pop(0)
        children = [(level + 1, child) for child in list(element)]
        if children:
            element.text = '\n' + indent * (level+1)  # for child open
        if queue:
            element.tail = '\n' + indent * queue[0][0]  # for sibling open
        else:
            element.tail = '\n' + indent * (level-1)  # for parent close
        queue[0:0] = children  # prepend so children come before siblings

def save_automata_to_files(automata_list):
    file_paths = []
    #base_path = '/path/to/save/location/'  # Specify your local path here

    for automaton in automata_list:
        root = ET.Element("structure")
        #root.set("type", "fa")
        #root.set("id", "1")
        comment = ET.Comment('Created with JFLAP 6.4.')
        root.append(comment)
        type_tag = ET.SubElement(root, "type")
        type_tag.text = "fa"
        automaton_tag = ET.SubElement(root, "automaton")
        
        # Adding states
        for i, state in enumerate(automaton['States']):
            state_tag = ET.SubElement(automaton_tag, "state")
            state_tag.set("id", str(i))
            state_tag.set("name", state)
            x, y = random.randint(100, 400), random.randint(100, 400)  # Random coordinates
            ET.SubElement(state_tag, "x").text = str(x)
            ET.SubElement(state_tag, "y").text = str(y)
            if state in automaton['Accept States']:
                ET.SubElement(state_tag, "final")
            if state == automaton['Start State']:
                ET.SubElement(state_tag, "initial")
        
        # Adding transitions
        state_id_map = {state: i for i, state in enumerate(automaton['States'])}
        for state, transitions in automaton['Transitions'].items():
            for input_symbol, target_state in transitions.items():
                transition_tag = ET.SubElement(automaton_tag, "transition")
                ET.SubElement(transition_tag, "from").text = str(state_id_map[state])
                ET.SubElement(transition_tag, "to").text = str(state_id_map[target_state])
                ET.SubElement(transition_tag, "read").text = input_symbol

        # Convert to ElementTree and write to XML
        prettify(root)

        tree = ET.ElementTree(root)
        file_name = f"{automaton['Name']}.jff"
        file_path = f"{file_name}"
        #tree.write(file_path)
        #file_paths.append(file_path)
        with open(file_path, "w", encoding="UTF-8") as file:
            file.write('<?xml version="1.0" encoding="UTF-8" standalone="no"?>\n')
            tree.write(file, encoding="unicode")
        file_paths.append(file_path)

    return file_paths

# Assuming 'automata_list' is already defined and contains your automata data
file_paths = save_automata_to_files(automata_list)
for path in file_paths:
    print(path)


Automaton_1.jff
Automaton_2.jff
Automaton_3.jff
Automaton_4.jff
Automaton_5.jff
Automaton_6.jff
Automaton_7.jff
Automaton_8.jff
Automaton_9.jff
Automaton_10.jff


In [29]:
import xml.etree.ElementTree as ET

def parse_jff_to_dict(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    automaton_dict = {
        'Name': 'Automaton',  # Default name, update if needed
        'States': [],
        'Alphabet': set(),  # To collect unique symbols
        'Transitions': {},
        'Start State': None,
        'Accept States': []
    }

    # Extract states and organize transitions
    for state in root.findall('.//state'):
        state_id = state.get('id')
        state_name = state.get('name') if state.get('name') else state_id
        automaton_dict['States'].append(state_name)
        automaton_dict['Transitions'][state_name] = {}

        # Check for initial and final tags
        if state.find('initial') is not None:
            automaton_dict['Start State'] = state_name
        if state.find('final') is not None:
            automaton_dict['Accept States'].append(state_name)

    # Extract transitions and form the transition dictionary properly
    for transition in root.findall('.//transition'):
        from_state = root.find(f".//state[@id='{transition.find('from').text}']").get('name')
        to_state = root.find(f".//state[@id='{transition.find('to').text}']").get('name')
        read_symbol = transition.find('read').text if transition.find('read') is not None else ''
        automaton_dict['Alphabet'].add(read_symbol)
        if read_symbol not in automaton_dict['Transitions'][from_state]:
            automaton_dict['Transitions'][from_state][read_symbol] = to_state
        else:
            # Handle non-deterministic cases by choosing one transition
            automaton_dict['Transitions'][from_state][read_symbol] = to_state

    # Convert Alphabet set to sorted list
    automaton_dict['Alphabet'] = sorted(automaton_dict['Alphabet'])

    return automaton_dict

# Example usage:
file_path = 'Automaton_1.jff'
automaton = parse_jff_to_dict(file_path)
automaton


{'Name': 'Automaton',
 'States': ['s0', 's1', 's2', 's3', 's4', 's5'],
 'Alphabet': ['a', 'b', 'c'],
 'Transitions': {'s0': {'a': 's0', 'b': 's2', 'c': 's5'},
  's1': {'a': 's1', 'b': 's3', 'c': 's0'},
  's2': {'a': 's2', 'b': 's4', 'c': 's1'},
  's3': {'a': 's3', 'b': 's5', 'c': 's2'},
  's4': {'a': 's4', 'b': 's0', 'c': 's3'},
  's5': {'a': 's5', 'b': 's5', 'c': 's5'}},
 'Start State': 's0',
 'Accept States': ['s4', 's5']}

In [32]:
class AutomatonMinimization:
    def __init__(self, automaton):
        self.states = {state: {'is_final': state in automaton['Accept States'], 'transitions': automaton['Transitions'][state]} for state in automaton['States']}
        self.start_state = automaton['Start State']
        self.alphabet = automaton['Alphabet']

    def minimize(self):
        partition = self.initial_partition()
        
        changed = True
        while changed:
            new_partition = []
            changed = False
            
            for group in partition:
                new_groups = {}
                for state in group:
                    state_transitions = {symbol: tuple(self.find_group(self.states[state]['transitions'].get(symbol, None), partition)) for symbol in self.alphabet}
                    group_key = tuple(sorted((symbol, trans) for symbol, trans in state_transitions.items()))
                    if group_key not in new_groups:
                        new_groups[group_key] = []
                    new_groups[group_key].append(state)
                
                if len(new_groups) > 1:
                    changed = True
                
                new_partition.extend(new_groups.values())

            partition = new_partition

        new_states = []
        for group in partition:
            is_final = any(self.states[state]['is_final'] for state in group)
            transitions = {}
            sample_state = group[0]
            for symbol in self.alphabet:
                target = self.states[sample_state]['transitions'].get(symbol)
                if target is not None:
                    transitions[symbol] = self.find_group(target, partition)[0]
            new_states.append({'is_final': is_final, 'transitions': transitions})

        # Reconstructing the minimized automaton
        return {
            'States': ['s{}'.format(i) for i in range(len(new_states))],
            'Alphabet': self.alphabet,
            'Transitions': {f's{i}': new_states[i]['transitions'] for i in range(len(new_states))},
            'Start State': 's0',
            'Accept States': ['s{}'.format(i) for i, state in enumerate(new_states) if state['is_final']]
        }

    def initial_partition(self):
        finais = {state for state in self.states if self.states[state]['is_final']}
        nao_finais = set(self.states) - finais
        return [finais, nao_finais] if finais and nao_finais else [finais]

    def find_group(self, state, partition):
        for group in partition:
            if state in group:
                return group
        return None


# Example usage:
automaton = {
    'Name': 'Automaton',
    'States': ['s0', 's1', 's2', 's3', 's4', 's5'],
    'Alphabet': ['a', 'b', 'c'],
    'Transitions': {
        's0': {'a': 's0', 'b': 's2', 'c': 's5'},
        's1': {'a': 's1', 'b': 's3', 'c': 's0'},
        's2': {'a': 's2', 'b': 's4', 'c': 's1'},
        's3': {'a': 's3', 'b': 's5', 'c': 's2'},
        's4': {'a': 's4', 'b': 's0', 'c': 's3'},
        's5': {'a': 's5', 'b': 's5', 'c': 's5'}
    },
    'Start State': 's0',
    'Accept States': ['s4', 's5']
}

minimizer = AutomatonMinimization(automaton)
minimized_automaton = minimizer.minimize()
minimized_automaton


{'States': ['s0', 's1', 's2', 's3', 's4', 's5'],
 'Alphabet': ['a', 'b', 'c'],
 'Transitions': {'s0': {'a': 's4', 'b': 's0', 'c': 's3'},
  's1': {'a': 's5', 'b': 's5', 'c': 's5'},
  's2': {'a': 's1', 'b': 's3', 'c': 's0'},
  's3': {'a': 's3', 'b': 's5', 'c': 's2'},
  's4': {'a': 's2', 'b': 's4', 'c': 's1'},
  's5': {'a': 's0', 'b': 's2', 'c': 's5'}},
 'Start State': 's0',
 'Accept States': ['s0', 's1']}

In [37]:
def minimize_dfa(automaton):
    states = set(automaton['States'])
    alphabet = automaton['Alphabet']
    transitions = automaton['Transitions']
    start_state = automaton['Start State']
    accept_states = set(automaton['Accept States'])

    # Initial partition
    P = [accept_states, states - accept_states]
    work_list = list(P)  # To keep track of partitions to refine

    # Refinement step
    while work_list:
        A = work_list.pop()
        for c in alphabet:
            X = set(s for s in states if transitions.get(s, {}).get(c) in A)
            new_partitions = []
            for Y in P:
                intersect = Y & X
                difference = Y - X
                if intersect and difference:
                    new_partitions.extend([intersect, difference])
                    if Y in work_list:
                        work_list.extend([intersect, difference])
                    else:
                        # Add the smaller partition to reduce future work
                        work_list.append(intersect if len(intersect) <= len(difference) else difference)
                else:
                    new_partitions.append(Y)
            P = new_partitions

    # Reconstructing the minimized DFA
    state_mapping = {}
    new_states = []
    new_transitions = {}
    new_start_state = None
    new_accept_states = []

    # Create new state names and find the new start and accept states
    for index, part in enumerate(P):
        new_state_name = f"q{index}"
        new_states.append(new_state_name)
        state_mapping.update({s: new_state_name for s in part})
        if start_state in part:
            new_start_state = new_state_name
        if part & accept_states:
            new_accept_states.append(new_state_name)

    # Reconstructing transitions for the new states
    for part in P:
        representative = next(iter(part))  # Any state from the partition
        new_state_name = state_mapping[representative]
        for a in alphabet:
            target_state = transitions[representative].get(a)
            if target_state:
                new_transitions.setdefault(new_state_name, {})[a] = state_mapping[target_state]

    # The minimized automaton in the same format
    minimized_automaton = {
        'States': new_states,
        'Alphabet': alphabet,
        'Transitions': new_transitions,
        'Start State': new_start_state,
        'Accept States': new_accept_states
    }

    return minimized_automaton

# Example DFA
# dfa = {
#     'States': ['s0', 's1', 's2', 's3', 's4', 's5'],
#     'Alphabet': ['a', 'b', 'c'],
#     'Transitions': {
#         's0': {'a': 's4', 'b': 's0', 'c': 's3'},
#         's1': {'a': 's5', 'b': 's5', 'c': 's5'},
#         's2': {'a': 's1', 'b': 's3', 'c': 's0'},
#         's3': {'a': 's3', 'b': 's5', 'c': 's2'},
#         's4': {'a': 's2', 'b': 's4', 'c': 's1'},
#         's5': {'a': 's0', 'b': 's2', 'c': 's5'}
#     },
#     'Start State': 's0',
#     'Accept States': ['s0', 's1']
# }
non_minimal_dfa = {
    'States': ['s0', 's1', 's2', 's3'],  # More states than necessary
    'Alphabet': ['a', 'b'],
    'Transitions': {
        's0': {'a': 's1', 'b': 's2'},
        's1': {'a': 's1', 'b': 's1'},  # States s1 and s3 can be merged
        's2': {'a': 's1', 'b': 's2'},
        's3': {'a': 's1', 'b': 's1'},  # Redundant state, behaves like s1
    },
    'Start State': 's0',
    'Accept States': ['s1']
}
non_minimal_dfa2 = {
    'States': ['s0', 's1', 's2', 's3'],  # Extra state s3 that is not necessary
    'Alphabet': ['a', 'b'],
    'Transitions': {
        's0': {'a': 's1', 'b': 's0'},
        's1': {'a': 's0', 'b': 's1'},
        's2': {'a': 's3', 'b': 's2'},  # s2 is an unnecessary duplicate of s0
        's3': {'a': 's2', 'b': 's3'},  # s3 is an unnecessary duplicate of s1
    },
    'Start State': 's0',
    'Accept States': ['s0']
} #deu ruim esse aqui
non_minimal_dfa3 = {
    'States': ['s0', 's1', 's2', 's3', 's4'],  # Extra states s3 and s4
    'Alphabet': ['a', 'b'],
    'Transitions': {
        's0': {'a': 's1', 'b': 's2'},
        's1': {'a': 's1', 'b': 's2'},
        's2': {'a': 's1', 'b': 's2'},
        's3': {'a': 's1', 'b': 's2'},  # s3 is redundant, behaves like s1
        's4': {'a': 's1', 'b': 's2'},  # s4 is redundant, behaves like s2
    },
    'Start State': 's0',
    'Accept States': ['s2']
}

minimized_dfa = minimize_dfa(non_minimal_dfa3)
minimized_dfa


{'States': ['q0', 'q1'],
 'Alphabet': ['a', 'b'],
 'Transitions': {'q0': {'a': 'q1', 'b': 'q0'}, 'q1': {'a': 'q1', 'b': 'q0'}},
 'Start State': 'q1',
 'Accept States': ['q0']}

In [40]:
def minimize_dfa(automaton):
    states = set(automaton['States'])
    alphabet = automaton['Alphabet']
    transitions = automaton['Transitions']
    start_state = automaton['Start State']
    accept_states = set(automaton['Accept States'])

    # Initial partition
    P = [accept_states, states - accept_states]
    t = len(P)  # Counter for partitions
    W = list(P)  # Working list to process partitions

    # Refinement step
    while W:
        A = W.pop(0)
        for a in alphabet:
            for i in range(len(P)):
                Qi = P[i]
                affected_states = {q for q in Qi if transitions.get(q, {}).get(a) in A}
                if not affected_states:
                    continue  # No states affected by this transition, skip to next

                # Compare all possible partitions
                for j1 in range(t):
                    for j2 in range(j1 + 1, t):
                        Qj1 = P[j1]
                        Qj2 = P[j2]
                        S1 = {q for q in Qi if transitions.get(q, {}).get(a) in Qj1}
                        S2 = {q for q in Qi if transitions.get(q, {}).get(a) in Qj2}

                        if S1 and S2:
                            if len(S1) <= len(S2):
                                new_partition = S1
                            else:
                                new_partition = S2

                            # Update Qi and add new partition
                            Qi_difference = Qi - new_partition
                            if Qi_difference:
                                P.append(Qi_difference)
                                if Qi in W:
                                    W.append(Qi_difference)
                                Qi.clear()
                                Qi.update(new_partition)
                                if new_partition not in W:
                                    W.append(new_partition)
                            t += 1  # Increment partition count

    # Construct minimized DFA
    state_mapping = {q: f"q{P.index(part)}" for part in P for q in part}
    new_states = [f"q{index}" for index, _ in enumerate(P)]
    new_transitions = {}
    new_start_state = state_mapping[start_state]
    new_accept_states = [state_mapping[q] for q in accept_states if q in state_mapping]

    # Reconstruct transitions for new states
    for q in states:
        original_state = state_mapping[q]
        new_transitions.setdefault(original_state, {})
        for a in alphabet:
            target_state = transitions[q].get(a)
            if target_state in state_mapping:
                new_transitions[original_state][a] = state_mapping[target_state]

    # Construct the output DFA
    minimized_dfa = {
        'States': new_states,
        'Alphabet': alphabet,
        'Transitions': new_transitions,
        'Start State': new_start_state,
        'Accept States': new_accept_states
    }

    return minimized_dfa

# Define a sample DFA to minimize
sample_dfa = {
    'States': ['s0', 's1', 's2', 's3'],
    'Alphabet': ['a', 'b'],
    'Transitions': {
        's0': {'a': 's1', 'b': 's2'},
        's1': {'a': 's2', 'b': 's3'},
        's2': {'a': 's3', 'b': 's0'},
        's3': {'a': 's0', 'b': 's1'}
    },
    'Start State': 's0',
    'Accept States': ['s3']
}

minimized_dfa = minimize_dfa(non_minimal_dfa3)
minimized_dfa


{'States': ['q0', 'q1'],
 'Alphabet': ['a', 'b'],
 'Transitions': {'q1': {'a': 'q1', 'b': 'q0'}, 'q0': {'a': 'q1', 'b': 'q0'}},
 'Start State': 'q1',
 'Accept States': ['q0']}